<a href="https://colab.research.google.com/github/danialebrat/Comp_8740/blob/master/Assignment_3/Information_gain_Danial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install sklearn_evaluation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.6 MB/s 
     |████████████████████████████████| 1.5 MB 10.7 MB/s 
     |████████████████████████████████| 96 kB 5.1 MB/s 
     |████████████████████████████████| 843 kB 56.3 MB/s 
     |████████████████████████████████| 1.6 MB 14.0 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [3]:
pip install mrmr_selection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 664 kB/s 
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=d5f551efd31e62334c3651b251530da2d548fc2830cd78331b433d2d0ef90ae6
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import chi2, SelectPercentile, mutual_info_classif, SelectKBest
from sklearn.metrics import confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn_evaluation.plot import grid_search
from mrmr import mrmr_classif

In [50]:
# Constant variables

DATASET = "/content/drive/MyDrive/Breastcancer.csv"

PLOT_PATH = "/content/drive/MyDrive/Plots/"

RESULT_PATH = "/content/drive/MyDrive/Results/"

In [55]:
class ML_Methods:

    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset


    def preprocess(self, df):
        """
        create x and y from a pandas dataframe
        x, which are 2D point will be scaled using min-max scaler
        :param dataframe:
        :return (Scaled X (minmax), y):
        """

        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values

        scaler = MinMaxScaler()
        x = scaler.fit_transform(X)

        return x, y

    def adding_methods(self):
        """
        adding all the methods with their specific names in a list
        :return: a List containing tuple of models (name of the model, model)
        """

        Models = []

        # models
        # Models.append(self.SVM_rbf())
        Models.append(self.Random_Forest())
        # Models.append(self.Deep_Neural_Network())

        return Models


    def Kfold_report(self, Models, x_train, y_train, dataset_name):
        """
        training all the models from the list of models using 10 fold cross validation
        :param x_train:
        :param y_train:
        :return:
        """

        print("**********")
        print("{} Dataset Results: ".format(dataset_name))

        results = []
        method_names = []
        for name, model in Models:
            # train the models
            KFold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
            CrossValidation = cross_val_score(model, x_train, y_train, cv=KFold, scoring='accuracy')
            results.append(CrossValidation)
            method_names.append(name)
            print(f"{name} Training Accuracy : {CrossValidation.mean()*100:.2f}%")

        return results, method_names


    def training_models(self, Models, x_train, x_test, y_train, y_test, datasetname):

        Results = []
        for name, model in Models:
            model.fit(x_train, y_train)
            predicted = model.predict(x_test)
            cm = confusion_matrix(y_test, predicted)
            AS = accuracy_score(y_test, predicted)

            Classifier, PPV, NPV, Sensitivity, Specificity, Testing_Accuracy = self.confusion_metrics(cm, AS, name, datasetname)
            self.plotting_confusion_matrix(cm, x_test, y_test, name)

            Results.append([Classifier,
                            PPV,
                            NPV,
                            Sensitivity,
                            Specificity,
                            Testing_Accuracy])

        # Storing the result
        self.storing_results(Results, datasetname)


    def Random_Forest(self):
        """
        create a Random_Forest classifier
        :return (name of the mode, Random_Forest model):
        """
        name = "Random_Forest"
        RF_Model = RandomForestClassifier()
        return (name, RF_Model)

    
    def SVM_rbf(self):
      """
      create a SVM classifier
      :return (name of the mode, QDA model):
      """
      name = "SVM_rbf"
      SVM_rbf_model = SVC(kernel = 'rbf', random_state = 42)
      return(name, SVM_rbf_model)


    def Keras_Deep_Neural_Network(self):
        """
        create a Deep_Neural_Network classifier
        """

        DNN_model = Sequential()

        DNN_model.add(Dense(64, activation='relu', input_dim=204))
        DNN_model.add(Dropout(0.1))
        DNN_model.add(Dense(32, activation='relu'))
        DNN_model.add(Dropout(0.2))
        DNN_model.add(Dense(16, activation='relu'))
        DNN_model.add(Dropout(0.2))
        DNN_model.add(Dense(32, activation='relu'))
        DNN_model.add(Dropout(0.2))
        DNN_model.add(Dense(64, activation='relu'))
        DNN_model.add(Dropout(0.3))

        DNN_model.add(Dense(5, activation='softmax'))

        DNN_model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return DNN_model


    def Deep_Neural_Network(self):
        """
        create a Deep_Neural_Network classifier
        :return (name of the mode, Deep_Neural_Network model):
        """
        name = "Deep_Neural_Network"

        # Wrap Keras model so it can be used by scikit-learn
        model = KerasClassifier(build_fn=self.Keras_Deep_Neural_Network,
                                epochs=20,
                                batch_size=16,
                                verbose=0)

        return (name, model)



    def data_spliting(self, x, y, test_size=0.2, random_state=42):
        """
        Split the data into x_train, x_test, y_train, y_test
        :param x: x (data)
        :param y: y (labels)
        :param test_size: size of test dataset
        :param random_state: 1 or 0
        :return: x_train, x_test, y_train, y_test
        """
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)
        return x_train, x_test, y_train, y_test

    def plotting(self, results, names, dataset_name):

        plt.figure(figsize=(12, 10))
        boxplot = plt.boxplot(results, patch_artist=True, labels=names)


        # fill with colors
        colors = ['pink', 'lightblue', 'lightgreen', 'lime', 'grey']
        for box, color in zip(boxplot['boxes'], colors):
            box.set(color=color)

        title = "Classifiers Comparison _ {}".format(dataset_name)
        plt.title(title)

        # saving the plots
        fname = PLOT_PATH + title + ".png"
        plt.savefig(fname, dpi=100)
        plt.close('all')


    def confusion_metrics(self, cm, accuracy_score, method_name, dataset_name):

        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
 
        # Sensitivity, hit rate, recall, or true positive rate
        TPR = TP/(TP+FN)
        # Specificity or true negative rate
        TNR = TN/(TN+FP) 
        # Precision or positive predictive value
        PPV = TP/(TP+FP)
        # Negative predictive value
        NPV = TN/(TN+FN)
        # Fall out or false positive rate
        FPR = FP/(FP+TN)
        # False negative rate
        FNR = FN/(TP+FN)
        # False discovery rate
        FDR = FP/(TP+FP)

        # Overall accuracy
        ACC = (TP+TN)/(TP+FP+FN+TN)

        avg_PPV  = np.average(PPV, axis=None, weights=None, returned=False)
        avg_NPV  = np.average(NPV, axis=None, weights=None, returned=False)
        avg_TPR  = np.average(TPR, axis=None, weights=None, returned=False)
        avg_TNR  = np.average(TNR, axis=None, weights=None, returned=False)
        avg_ACC  = np.average(ACC, axis=None, weights=None, returned=False)

        # Accuracy for test set
        accuracy = round(accuracy_score*100, 2)

        print("**************")
        print("Classifier: {} _ Dataset: {}".format(method_name, dataset_name))
        print("PPV:{:.2f} | NPV:{:.2f} | Sensitivity:{:.2f} | Specificity:{:.2f} | Total_Accuracy:{:.2f}".format(avg_PPV, avg_NPV, avg_TPR, avg_TNR, avg_ACC))
        print("Accuracy Score for test_set: {:.2f} ".format(accuracy))

        return method_name, avg_PPV, avg_NPV, avg_TPR, avg_TNR, accuracy



    def plotting_confusion_matrix(self, cm, x_test, y_test, name):
      
      plot_confusion_matrix(cm, x_test, y_test)  
      title = "Confusion Matrix _ {}".format(name)
      plt.title(title)
      plt.show()

      # saving the plots
      fname = PLOT_PATH + title + ".png"
      plt.savefig(fname, dpi=100)
      plt.close('all')



    def storing_results(self, Results , dataset_name):
        """
        Storing the results in a csv file
        :return:
        """

        info = pd.DataFrame(Results, columns=['Classifier', 'PPV', 'NPV', 'Sensitivity', 'Specificity','Testing_Accuracy'])
        self.store(info, dest=RESULT_PATH, name=dataset_name)


    def store(self, df, dest, name):
        """
        Storing the results as an excel file in a folder
        :param dest:
        :param name:
        :return:
        """
        path = dest + name + ".xlsx"
        df.to_excel(path)

In [7]:
def read_csv(file):
  # read dataset csv as a Dataframe
  df = pd.read_csv(file)
  return df

In [8]:
def grid_search_tuning(Models, x_train, y_train):

  for name, model in Models:
    if name == "SVM_rbf":
      
      continue
      C_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
      gamma_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
      degrees = range(3, 100, 5)
      parameters = dict(gamma=gamma_range, C=C_range, degree=degrees)

    else:
      parameters = {
          'max_features' : [7, 8, 9, 10, 12, 13, 14, 15, 20],
          'n_estimators': [250, 300, 350, 400, 450, 500, 550],
          }


    grid = GridSearchCV(model, parameters, refit = True, verbose = 0)
  
    # fitting the model for grid search
    grid.fit(x_train, y_train)


    gs = GridSearchCV(estimator = model,
                              param_grid = parameters,
                              scoring = 'accuracy',
                              cv = 10,
                              refit = True,
                              n_jobs = -1,
                              verbose = 0)
    
    gs.fit(x_train, y_train)

    if name == "Random_Forest":
      grid_search(gs.cv_results_, change='max_features')
      title = "Progress of the GridSearch _ {}".format(name)
      plt.title(title)
      plt.show()

      # saving the plots
      fname = PLOT_PATH + title + ".png"
      plt.savefig(fname, dpi=100)
      plt.close('all')

    best_accuracy = gs.best_score_
    best_parameters = gs.best_params_

    print("{} Classifies: ***** ".format(name))
    print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
    print("Best Parameters:", best_parameters)

In [9]:
def feature_selection_Information_gain(x, y):

  scores = mutual_info_classif(x,y)
  
  # p_values = pd.Series(scores)
  # p_values.sort_values(ascending = False , inplace = True)
  # p_values.plot.bar()
  # plt.show()

  X_new = SelectKBest(mutual_info_classif, k=100).fit_transform(x, y)
  return X_new


In [56]:
def feature_selection_mrmr(X, y):
  
  X_new1 = pd.DataFrame(X, columns = X.columns)

  scores = mrmr_classif(X_new1,y,K = 30)

  X_new = X[scores]

  # X_new = SelectKBest(mrmr_classif(X_new1,y,K = 25), k=25).fit_transform(X_new1, y)
  
  # p_values = pd.Series(X_new, index=scores)
  # p_values.sort_values(ascending = False , inplace = True)
  # p_values.plot.bar()
  # title = "Feature Selection _ MRMR"
  # plt.title(title)
  # plt.show()

  # # saving the plots
  # fname = PLOT_PATH + title + ".png"
  # plt.savefig(fname, dpi=100)
  # plt.show()
  # plt.close('all')

  return X_new

In [29]:
def feature_selection_ChiSquare(x, y):

  scores = chi2(x,y)
  
  # p_values = pd.Series(scores)
  # p_values.sort_values(ascending = False , inplace = True)
  # p_values.plot.bar()
  # plt.show()

  X_new = SelectPercentile(chi2, percentile=85).fit_transform(x, y)

  return X_new

In [57]:
if __name__ == "__main__":

    name = "Breast Cancer"

    # reading the data
    Data = read_csv(DATASET)
    
    # creating and ML_method object for the dataset
    method = ML_Methods(name="BreastCancer", dataset=Data)
    X, y = method.preprocess(Data)

    # Applying feature selection
    x_filtering  = feature_selection_ChiSquare(X, y)
    x_filtering  = feature_selection_Information_gain(x_filtering, y)
    x_filtering_df = pd.DataFrame(x_filtering)
    x =  feature_selection_mrmr(x_filtering_df, y)

    print('Original number of features:', X.shape)
    print('Reduced number of features:', x.shape)

    

    # spliting the dataset
    x_train, x_test, y_train, y_test = method.data_spliting(x, y)

    # Adding methods:
    Models = method.adding_methods()

    # train the models and getting the results
    results, method_name = method.Kfold_report(Models, x_train, y_train, name)

    method.plotting(results, method_name, name)

    method.training_models(Models, x_train, x_test, y_train, y_test, name)


    print("******************************")


    grid_search_tuning(Models, x_train, y_train)




100%|██████████| 30/30 [00:01<00:00, 20.91it/s]


Original number of features: (158, 13582)
Reduced number of features: (158, 30)
**********
Breast Cancer Dataset Results: 
Random_Forest Training Accuracy : 86.41%
**************
Classifier: Random_Forest _ Dataset: Breast Cancer
PPV:0.82 | NPV:0.96 | Sensitivity:0.84 | Specificity:0.96 | Total_Accuracy:0.94
Accuracy Score for test_set: 84.38 


ValueError: ignored